In [3]:
import lancedb
import pandas as pd
import json
from pathlib import Path


In [6]:

# 设置数据库路径
db_path = "../lancedb"

print("🔗 连接到 LanceDB 数据库...")
try:
    db = lancedb.connect(db_path)
    print(f"✅ 成功连接到数据库: {db_path}")
except Exception as e:
    print(f"❌ 连接失败: {e}")
    exit()


🔗 连接到 LanceDB 数据库...
✅ 成功连接到数据库: ../lancedb


In [7]:

# 获取所有表名
table_names = db.table_names()
print(f"\n📊 发现 {len(table_names)} 个表:")
for name in table_names:
    print(f"   - {name}")



📊 发现 3 个表:
   - default-community-full_content
   - default-entity-description
   - default-text_unit-text


In [8]:

# 加载所有表
tables = {}
for table_name in table_names:
    try:
        tables[table_name] = db.open_table(table_name)
        print(f"✅ 成功加载表: {table_name}")
    except Exception as e:
        print(f"❌ 加载表 {table_name} 失败: {e}")

print("\n" + "="*50)


✅ 成功加载表: default-community-full_content
✅ 成功加载表: default-entity-description
✅ 成功加载表: default-text_unit-text



In [9]:

# 分析每个表的基本信息
for table_name, table in tables.items():
    print(f"\n📋 分析表: {table_name}")
    print("-" * 30)
    
    try:
        # 获取行数
        row_count = table.count_rows()
        print(f"行数: {row_count:,}")
        
        # 获取表结构
        schema = table.schema
        print(f"列数: {len(schema)}")
        print("列信息:")
        for field in schema:
            print(f"  - {field.name}: {field.type}")
        
        # 获取前5行样本数据
        print("\n前5行样本数据:")
        sample_data = table.head(5).to_pandas()
        
        # 如果有attributes列，尝试解析JSON
        if 'attributes' in sample_data.columns:
            try:
                sample_data['attributes_preview'] = sample_data['attributes'].apply(
                    lambda x: str(json.loads(x))[:100] + "..." if isinstance(x, str) and len(str(json.loads(x))) > 100 
                    else json.loads(x) if isinstance(x, str) else x
                )
            except:
                pass
        
        # 显示非向量列的数据
        display_cols = [col for col in sample_data.columns if col != 'vector']
        if display_cols:
            for i, row in sample_data.iterrows():
                print(f"\n第 {i+1} 行:")
                for col in display_cols[:4]:  # 只显示前4列
                    value = str(row[col])
                    if len(value) > 100:
                        value = value[:100] + "..."
                    print(f"  {col}: {value}")
                if len(display_cols) > 4:
                    print(f"  ... 还有 {len(display_cols)-4} 列")
        
    except Exception as e:
        print(f"❌ 分析表失败: {e}")

print("\n" + "="*50)



📋 分析表: default-community-full_content
------------------------------
行数: 8
列数: 4
列信息:
  - id: string
  - text: string
  - vector: fixed_size_list<item: float>[1024]
  - attributes: string

前5行样本数据:

第 1 行:
  id: 2e0510242df24ca39c15d796d3d47bfa
  text: # 贾府寿宴的社交网络

该社区围绕贾珍及其在贾敬寿辰宴会中的角色展开，展示了贾府内部的家族关系、社交礼仪和权力结构。贾珍作为宁国府的当家，负责筹备宴会，与家族成员、仆人及宴会各项准备活动有着密切的联...
  attributes: {"title": "# \u8d3e\u5e9c\u5bff\u5bb4\u7684\u793e\u4ea4\u7f51\u7edc\n\n\u8be5\u793e\u533a\u56f4\u7ed...
  attributes_preview: {'title': '# 贾府寿宴的社交网络\n\n该社区围绕贾珍及其在贾敬寿辰宴会中的角色展开，展示了贾府内部的家族关系、社交礼仪和权力结构。贾珍作为宁国府的当家，负责筹备宴会，与家族成员、仆人及宴...

第 2 行:
  id: a31f56bf9ffd42109dbe711e7f518dcf
  text: # 贾府中的权力与关系网络

该社区聚焦于贾府内部的关键人物及其复杂的关系网络，特别是凤姐儿和王夫人在家族中的权力和地位。凤姐儿作为贾琏之妻和贾府的实际掌权者之一，与王夫人之间的婆媳关系，以及她与贾珍...
  attributes: {"title": "# \u8d3e\u5e9c\u4e2d\u7684\u6743\u529b\u4e0e\u5173\u7cfb\u7f51\u7edc\n\n\u8be5\u793e\u533...
  attributes_preview: {'title': '# 贾府中的权力与关系网络\n\n该社区聚焦于贾府内部的关键人物及其复杂的关系网络，特别是凤姐儿和王夫人在家族中的权力和地位。凤姐儿作为贾琏之妻和贾府的实际掌权者之一，与王夫人